Bibliotecas

In [1]:
import pandas as pd
import os
import datetime

Variaveis Globais

In [2]:
# Diretório contendo os arquivos
input_directory = 'DadosClimaticos/'
# Lista para armazenar os DataFrames de cada cidade
all_data = []

Processamento dos datasets 

In [3]:
# Função para processar cada arquivo
def process_city_files(file_path):

    # Lê o nome da cidade na primeira linha
    with open(file_path, 'r', encoding='utf-8') as f:
        city_name = f.readline().split(':',1)[1].strip() # Extract city name
        
        
    # Lê os dados a partir da linha 10
    data = pd.read_csv(
        file_path,
        sep=";",
        skiprows=9,
        encoding="utf-8"
    )    

    # Renomeia as colunas
    data.columns = [
        "Data Medida",
        "Precipitacao Total (mm)",
        "Temperatura Maxima (°C)",
        "Temperatura Media (°C)",
        "Temperatura Minima (°C)",
        "Umidade Relativa Media (%)",
        "Umidade Relativa Minima (%)",
        "Velocidade dos Ventos Media (m/s)",
        " "
    ]   
    # Remove colunas desnecessárias
    data.drop(columns=[' '], inplace=True)

    # Adiciona o nome da cidade
    data["Cidade"] = city_name

    # Retorna o nome da cidade
    return data


In [4]:
def process_files(input_directory):
    # Processa todos os arquivos no diretório
    for filename in os.listdir(input_directory):
        
        if filename.endswith('.csv'): # processa arquivos .csv 
            file_path = os.path.join(input_directory, filename)

            # Processa o arquivo da cidade
            data = process_city_files(file_path)
            
            # Adiciona os dados na lista
            all_data.append(data)
            
    return all_data

Tratamento dos Dados

In [5]:
# Dados dos Datasets
all_data = process_files(input_directory)

# Concatena todos os dados
data = pd.concat(all_data, ignore_index=True)

# Modifica o tipo da date para datetime
data['Data Medida'] = pd.to_datetime(data['Data Medida'], format='%Y-%m-%d')

In [6]:
# Dropa a ultima linha
data = data.drop(data.index[-1])

EDA - Análise Exploratória dos Dados

In [7]:
summary = data.describe()
summary

,Data Medida,Precipitacao Total (mm),Temperatura Maxima (°C),Temperatura Media (°C),Temperatura Minima (°C),Umidade Relativa Media (%),Umidade Relativa Minima (%),Velocidade dos Ventos Media (m/s)
count,27880,25676.000000,24066.000000,22335.000000,25457.000000,20889.000000,20815.000000,20298.000000
mean,2021-11-26 16:17:31.592539648,5.397542,31.250191,25.730486,21.894355,73.912399,62.781648,1.493861
min,2020-01-01 00:00:00,0.000000,8.500000,5.000000,-0.100000,6.000000,10.000000,0.000000
25%,2020-11-13 00:00:00,0.000000,29.300000,24.300000,20.200000,66.800000,51.000000,0.800000
50%,2021-11-10 00:00:00,0.000000,31.700000,26.700000,23.000000,76.300000,65.000000,1.300000
75%,2022-12-02 00:00:00,4.000000,33.700000,28.000000,24.400000,84.000000,76.000000,2.000000
max,2024-01-01 00:00:00,225.100000,44.200000,36.200000,30.200000,100.000000,100.000000,8.900000
std,NaN,13.146645,3.964993,3.549616,3.710638,14.278950,18.002811,1.056723


In [8]:
data.shape

(27880, 9)

In [9]:
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27880 entries, 0 to 27879
Data columns (total 9 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Data Medida                        27880 non-null  datetime64[ns]
 1   Precipitacao Total (mm)            25676 non-null  float64       
 2   Temperatura Maxima (°C)            24066 non-null  float64       
 3   Temperatura Media (°C)             22335 non-null  float64       
 4   Temperatura Minima (°C)            25457 non-null  float64       
 5   Umidade Relativa Media (%)         20889 non-null  float64       
 6   Umidade Relativa Minima (%)        20815 non-null  float64       
 7   Velocidade dos Ventos Media (m/s)  20298 non-null  float64       
 8   Cidade                             27880 non-null  object        
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 3.2 MB


In [10]:
data.duplicated().sum()

np.int64(0)

In [11]:
data.isna().sum()

Data Medida                             0
Precipitacao Total (mm)              2204
Temperatura Maxima (°C)              3814
Temperatura Media (°C)               5545
Temperatura Minima (°C)              2423
Umidade Relativa Media (%)           6991
Umidade Relativa Minima (%)          7065
Velocidade dos Ventos Media (m/s)    7582
Cidade                                  0
dtype: int64

In [12]:
# Função para preencher valores faltantes
def fill_missing_values(data):
    fill_methods = {
        "Precipitacao Total (mm)": "mean",  # Preenche com a média
        "Temperatura Maxima (°C)": "interpolate",  # Interpolação linear
        "Temperatura Media (°C)": "interpolate",  # Interpolação linear
        "Temperatura Minima (°C)": "median",  # Preenche com a mediana
        "Umidade Relativa Media (%)": "mean",  # Preenche com a média
        "Umidade Relativa Minima (%)": "median",  # Preenche com a mediana
        "Velocidade dos Ventos Media (m/s)": "mean"  # Preenche com a média
    }
    
    for column, method in fill_methods.items():
        if method == "mean":
            data[column] = data.groupby('Cidade')[column].transform(lambda x: x.fillna(x.mean()))
        elif method == "median":
            data[column] = data.groupby('Cidade')[column].transform(lambda x: x.fillna(x.median()))
        elif method == "interpolate":
            data[column] = data.groupby('Cidade')[column].transform(lambda x: x.interpolate(method='linear'))
    

    # Preencher faltantes restantes com forward fill ou backward fill
    data.ffill(inplace=True)
    data.bfill(inplace=True)
    return data

# Preenche os valores faltantes
data = fill_missing_values(data)


In [13]:
data.isna().sum()

Data Medida                          0
Precipitacao Total (mm)              0
Temperatura Maxima (°C)              0
Temperatura Media (°C)               0
Temperatura Minima (°C)              0
Umidade Relativa Media (%)           0
Umidade Relativa Minima (%)          0
Velocidade dos Ventos Media (m/s)    0
Cidade                               0
dtype: int64

In [14]:
data.describe()

,Data Medida,Precipitacao Total (mm),Temperatura Maxima (°C),Temperatura Media (°C),Temperatura Minima (°C),Umidade Relativa Media (%),Umidade Relativa Minima (%),Velocidade dos Ventos Media (m/s)
count,27880,27880.000000,27880.000000,27880.000000,27880.000000,27880.000000,27880.000000,27880.000000
mean,2021-11-26 16:17:31.592539648,5.362502,31.323626,25.745213,22.023447,73.472140,63.315890,1.291087
min,2020-01-01 00:00:00,0.000000,8.500000,5.000000,-0.100000,6.000000,10.000000,0.000000
25%,2020-11-13 00:00:00,0.000000,29.500000,24.200000,20.500000,66.500000,53.000000,0.400000
50%,2021-11-10 00:00:00,0.000000,31.600000,26.400000,23.100000,76.683431,66.000000,1.000000
75%,2022-12-02 00:00:00,5.035802,33.700000,27.800000,24.500000,84.500000,75.000000,1.800000
max,2024-01-01 00:00:00,225.100000,44.200000,36.200000,30.200000,100.000000,100.000000,8.900000
std,NaN,12.620507,3.841610,3.278819,3.625537,14.512750,16.622159,1.110995


In [15]:
data = data.loc[data['Data Medida'] != '2024-01-01']

In [16]:
data.to_csv('DadosClimaticosTransformados.csv')